# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [81]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    # YOUR CODE HERE
    df_nhl = pd.read_csv('assets/nhl.csv')
    df_nhl =df_nhl[df_nhl['year']==2018]
    patternDel = ".*Division"
    filter = df_nhl['team'].str.contains(patternDel)
    df_nhl =df_nhl[~filter]
    df_nhl.replace('\*','',regex = True,inplace = True)
    df_nhl.replace('^[^\s]+\s','',regex = True,inplace = True)
    #df_nhl.replace('bay','',regex = True,inplace = True)
    #df_nhl.replace('bay','',regex = True,inplace = True)
    pr= df_nhl['team'].copy()
    df_nhl['team'][1] = 'Lightning'
    df_nhl['team'][14] = 'Devils'
    df_nhl['team'][16] = 'Islanders'
    df_nhl['team'][17] = 'Rangers'
    df_nhl['team'][23] = 'Blues'
    df_nhl['team'][29] = 'Sharks'
    df_nhl['team'][30] = 'Kings'
    df_nhl['team'][3] = 'Leafs'
    df_nhl['team'][5] = 'Wings'
    df_nhl['team'][13] = 'Jackets'
    df_nhl['team'][27] = 'Knights'
    print(df_nhl['team']+pr)
    df_nhl['win-loss-ratio'] = pd.to_numeric(df_nhl['W']) /(pd.to_numeric(df_nhl['W'])+pd.to_numeric(df_nhl['L']))
    df_nhl = df_nhl[['team','win-loss-ratio']]
    #print(df_nhl)
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.drop(columns = {'B6','MLS','CFL'})
    #cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.replace('\[note \d*\]','',regex = True,inplace = True)
    #cities.replace('\[\d*\]','',regex = True,inplace = True)
    cities = cities[['Metropolitan area','Population (2016 est.)[8]','NHL']]
    cities['NHL'] = cities['NHL'].str.split(' ')
    print(cities.set_index('Metropolitan area').sort_index().index)
    cities = cities.explode('NHL')
    #print(cities)
    alll = df_nhl.set_index('team').join(cities.set_index('NHL'),how ='inner')
    popu = alll.set_index('Metropolitan area')['Population (2016 est.)[8]']
    popu = popu.drop_duplicates() 
    avg = alll.groupby('Metropolitan area')['win-loss-ratio'].mean()
    #avg = avg.join(cities.set_index('Metropolitan area',how = 'inner'))
    abc = avg.to_frame().join(popu.to_frame())
    print(abc.index)
    population_by_region = pd.to_numeric(abc['Population (2016 est.)[8]']) # pass in metropolitan area population from cities
    win_loss_by_region = abc['win-loss-ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nhl_correlation()

1      LightningBay Lightning
2                BruinsBruins
3            LeafsMaple Leafs
4            PanthersPanthers
5              WingsRed Wings
6          CanadiensCanadiens
7            SenatorsSenators
8                SabresSabres
10           CapitalsCapitals
11           PenguinsPenguins
12               FlyersFlyers
13        JacketsBlue Jackets
14        DevilsJersey Devils
15       HurricanesHurricanes
16    IslandersYork Islanders
17        RangersYork Rangers
19         PredatorsPredators
20                   JetsJets
21                   WildWild
22         AvalancheAvalanche
23           BluesLouis Blues
24                 StarsStars
25       BlackhawksBlackhawks
27      KnightsGolden Knights
28                 DucksDucks
29          SharksJose Sharks
30         KingsAngeles Kings
31               FlamesFlames
32               OilersOilers
33             CanucksCanucks
34             CoyotesCoyotes
Name: team, dtype: object
Index(['Atlanta', 'Baltimore', 'Boston', 'Bu

0.012486162921209909

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [97]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    df_nhl = pd.read_csv('assets/nba.csv')
    df_nhl =df_nhl[df_nhl['year']==2018]
    patternDel = ".*Division"
    filter = df_nhl['team'].str.contains(patternDel)
    df_nhl =df_nhl[~filter]
    df_nhl.replace('\*','',regex = True,inplace = True)

    df_nhl.replace('^[^\s]+\s','',regex = True,inplace = True)
    df_nhl.replace('\(\d*\)','',regex = True,inplace = True)
    df_nhl.replace('\s','',regex = True,inplace = True)

    #df_nhl.replace('bay','',regex = True,inplace = True)
    pr= df_nhl['team'].copy()
    df_nhl['team'][10] = 'Knicks'
    df_nhl['team'][16] = 'Warriors'
    df_nhl['team'][17] = 'Blazers'
    df_nhl['team'][18] = 'Thunder'
    df_nhl['team'][20] = 'Pelicans'
    df_nhl['team'][21] = 'Spurs'
    df_nhl['team'][24] = 'Clippers'
    df_nhl['team'][25] = 'Lakers'
    #print(df_nhl['team']+pr)
    df_nhl['win-loss-ratio'] = pd.to_numeric(df_nhl['W']) /(pd.to_numeric(df_nhl['W'])+pd.to_numeric(df_nhl['L']))
    df_nhl = df_nhl[['team','win-loss-ratio']]
    print(df_nhl)
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.drop(columns = {'B6','MLS','CFL'})
    #cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.replace('\[note \d*\]','',regex = True,inplace = True)
    #cities.replace('\[\d*\]','',regex = True,inplace = True)
    cities = cities[['Metropolitan area','Population (2016 est.)[8]','NBA']]
    cities['NBA'] = cities['NBA'].str.split(' ')
    #print(cities.set_index('Metropolitan area').sort_index().index)
    cities = cities.explode('NBA')
    print(cities)
    alll = df_nhl.set_index('team').join(cities.set_index('NBA'),how ='inner')
    print(alll)
    popu = alll.set_index('Metropolitan area')['Population (2016 est.)[8]']
    popu = popu.drop_duplicates() 
    avg = alll.groupby('Metropolitan area')['win-loss-ratio'].mean()
    #avg = avg.join(cities.set_index('Metropolitan area',how = 'inner'))
    abc = avg.to_frame().join(popu.to_frame())
    print(abc.index)
    population_by_region = pd.to_numeric(abc['Population (2016 est.)[8]']) # pass in metropolitan area population from cities
    win_loss_by_region = abc['win-loss-ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    print(len(population_by_region))

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    


nba_correlation()

            team  win-loss-ratio
0        Raptors        0.719512
1        Celtics        0.670732
2          76ers        0.634146
3      Cavaliers        0.609756
4         Pacers        0.585366
5           Heat        0.536585
6          Bucks        0.536585
7        Wizards        0.524390
8        Pistons        0.475610
9        Hornets        0.439024
10        Knicks        0.353659
11          Nets        0.341463
12         Bulls        0.329268
13         Magic        0.304878
14         Hawks        0.292683
15       Rockets        0.792683
16      Warriors        0.707317
17       Blazers        0.597561
18       Thunder        0.585366
19          Jazz        0.585366
20      Pelicans        0.585366
21         Spurs        0.573171
22  Timberwolves        0.573171
23       Nuggets        0.560976
24      Clippers        0.512195
25        Lakers        0.426829
26         Kings        0.329268
27     Mavericks        0.292683
28     Grizzlies        0.268293
29        

-0.17657160252844614

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [106]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    
    df_nhl = pd.read_csv('assets/mlb.csv')
    df_nhl =df_nhl[df_nhl['year']==2018]
    patternDel = ".*Division"
    filter = df_nhl['team'].str.contains(patternDel)
    df_nhl =df_nhl[~filter]
    df_nhl.replace('\*','',regex = True,inplace = True)
    df_nhl.replace('^[^\s]+\s','',regex = True,inplace = True)
    df_nhl.replace('\(\d*\)','',regex = True,inplace = True)
    #df_nhl.replace('bay','',regex = True,inplace = True)
    pr= df_nhl['team'].copy()
    print(df_nhl)
    #df_nhl['team'][1] = 'Sox'
    df_nhl['team'][1] = 'Yankees'
    df_nhl['team'][2] = 'Rays'
    df_nhl['team'][3] = 'Jays'
    df_nhl['team'][9] = 'Royals'
    df_nhl['team'][13] = 'Angels'
    df_nhl['team'][18] = 'Mets'
    df_nhl['team'][22] = 'Cardinals'
    df_nhl['team'][25] = 'Dodgers'
    df_nhl['team'][28] = 'Giants'
    df_nhl['team'][29] = 'Padres'
    df_nhl['team'][0] = 'RedSox'
    df_nhl['team'][8] = 'WhiteSox'
    #print(df_nhl['team']+pr)
    df_nhl['win-loss-ratio'] = pd.to_numeric(df_nhl['W']) /(pd.to_numeric(df_nhl['W'])+pd.to_numeric(df_nhl['L']))
    df_nhl = df_nhl[['team','win-loss-ratio']]
    print(df_nhl)
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.drop(columns = {'B6','MLS','CFL'})
    #cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.replace('\[note \d*\]','',regex = True,inplace = True)
    #cities.replace('\[\d*\]','',regex = True,inplace = True)
    cities = cities[['Metropolitan area','Population (2016 est.)[8]','MLB']]
    print(cities)
    cities['MLB'][7] = 'RedSox'
    cities['MLB'][3] = 'Cubs WhiteSox'
    print(cities)

    cities['MLB'] = cities['MLB'].str.split(' ')
    print(cities.set_index('Metropolitan area').sort_index().index)
    cities = cities.explode('MLB')
    print(cities)
    alll = df_nhl.set_index('team').join(cities.set_index('MLB'),how ='inner')
    popu = alll.set_index('Metropolitan area')['Population (2016 est.)[8]']
    popu = popu.drop_duplicates() 
    avg = alll.groupby('Metropolitan area')['win-loss-ratio'].mean()
    #avg = avg.join(cities.set_index('Metropolitan area',how = 'inner'))
    abc = avg.to_frame().join(popu.to_frame())
    print(abc.index)
    population_by_region = pd.to_numeric(abc['Population (2016 est.)[8]']) # pass in metropolitan area population from cities
    win_loss_by_region = abc['win-loss-ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    print(len(population_by_region))

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q1: There should be 28 teams being analysed for NHL"
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    


mlb_correlation()

                team    W    L   W-L%    GB  year League
0            Red Sox  108   54  0.667    --  2018    MLB
1       York Yankees  100   62  0.617   8.0  2018    MLB
2           Bay Rays   90   72  0.556  18.0  2018    MLB
3          Blue Jays   73   89  0.451  35.0  2018    MLB
4            Orioles   47  115  0.290  61.0  2018    MLB
5            Indians   91   71  0.562    --  2018    MLB
6              Twins   78   84  0.481  13.0  2018    MLB
7             Tigers   64   98  0.395  27.0  2018    MLB
8          White Sox   62  100  0.383  29.0  2018    MLB
9        City Royals   58  104  0.358  33.0  2018    MLB
10            Astros  103   59  0.636    --  2018    MLB
11         Athletics   97   65  0.599   6.0  2018    MLB
12          Mariners   89   73  0.549  14.0  2018    MLB
13    Angeles Angels   80   82  0.494  23.0  2018    MLB
14           Rangers   67   95  0.414  36.0  2018    MLB
15            Braves   90   72  0.556    --  2018    MLB
16         Nationals   82   80 

0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [120]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    
    df_nhl = pd.read_csv('assets/nfl.csv')
    df_nhl =df_nhl[df_nhl['year']==2018]
    patternDel = "AFC.*"
    filter = df_nhl['team'].str.contains(patternDel)
    df_nhl =df_nhl[~filter]
    patternDel = "NFC.*"
    filter = df_nhl['team'].str.contains(patternDel)
    df_nhl =df_nhl[~filter]
    print(df_nhl)
    df_nhl.replace('\*','',regex = True,inplace = True)
    df_nhl.replace('\+','',regex = True,inplace = True)

    df_nhl.replace('^[^\s]+\s','',regex = True,inplace = True)
    df_nhl.replace('\(\d*\)','',regex = True,inplace = True)
    #df_nhl.replace('bay','',regex = True,inplace = True)
    pr= df_nhl['team'].copy()
    df_nhl['team'][1] = 'Patriots'
    df_nhl['team'][4] = 'Jets'
    df_nhl['team'][16] = 'Chiefs'
    df_nhl['team'][17] = 'Chargers'
    df_nhl['team'][24] = 'Giants'
    df_nhl['team'][28] = 'Packers'
    df_nhl['team'][31] = 'Saints'
    df_nhl['team'][34] = 'Buccaneers'
    df_nhl['team'][36] = 'Rams'
    df_nhl['team'][38] = '49ers'

    print(df_nhl['team']+pr)
    df_nhl['win-loss-ratio'] = pd.to_numeric(df_nhl['W']) /(pd.to_numeric(df_nhl['W'])+pd.to_numeric(df_nhl['L']))
    df_nhl = df_nhl[['team','win-loss-ratio']]
    #print(df_nhl)
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.drop(columns = {'B6','MLS','CFL'})
    #cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.replace('\[note \d*\]','',regex = True,inplace = True)
    #cities.replace('\[\d*\]','',regex = True,inplace = True)
    cities = cities[['Metropolitan area','Population (2016 est.)[8]','NFL']]
    cities['NFL'] = cities['NFL'].str.split(' ')
    print(cities.set_index('Metropolitan area').sort_index().index)
    cities = cities.explode('NFL')
    #print(cities)
    alll = df_nhl.set_index('team').join(cities.set_index('NFL'),how ='inner')
    popu = alll.set_index('Metropolitan area')['Population (2016 est.)[8]']
    popu = popu.drop_duplicates() 
    avg = alll.groupby('Metropolitan area')['win-loss-ratio'].mean()
    #avg = avg.join(cities.set_index('Metropolitan area',how = 'inner'))
    abc = avg.to_frame().join(popu.to_frame())
    print(abc.index)
    population_by_region = pd.to_numeric(abc['Population (2016 est.)[8]']) # pass in metropolitan area population from cities
    win_loss_by_region = abc['win-loss-ratio'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    print(len(population_by_region))

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q1: There should be 28 teams being analysed for NHL"
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    


nfl_correlation()

    DSRS   L League    MoV  OSRS   PA    PD   PF    SRS   SoS  T   W  W-L%  \
1    2.1   5    NFL    6.9   3.1  325   111  436    5.2  -1.8  0  11  .688   
2   -5.2   9    NFL   -7.1  -3.6  433  -114  319   -8.8  -1.7  0   7  .438   
3   -0.6  10    NFL   -6.6  -6.3  374  -105  269   -6.9  -0.3  0   6  .375   
4   -5.9  12    NFL   -6.8  -2.0  441  -108  333   -7.8  -1.1  0   4  .250   
6    6.4   6    NFL    6.4   0.6  287   102  389    7.0   0.6  0  10  .625   
7    1.7   6    NFL    4.3   3.9  360    68  428    5.6   1.3  1   9  .594   
8    0.6   8    NFL   -2.1  -1.0  392   -33  359   -0.3   1.7  1   7  .469   
9   -3.4  10    NFL   -5.4   0.0  455   -87  368   -3.4   2.0  0   6  .375   
11   1.4   5    NFL    5.4   2.4  316    86  402    3.8  -1.5  0  11  .688   
12  -0.6   6    NFL    5.6   3.9  344    89  433    3.4  -2.2  0  10  .625   
13   3.5   7    NFL    0.4  -3.2  303     7  310    0.2  -0.2  0   9  .563   
14   4.0  11    NFL   -4.4  -8.1  316   -71  245   -4.0   0.4  0

0.004922112149349409

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values